In [0]:
# Create a sample table
spark.sql("""
CREATE TABLE IF NOT EXISTS sample_table (
    id INT,
    value STRING,
    updated_at TIMESTAMP
)
""")


In [0]:
%sql
select * from sample_table

In [0]:

# Insert initial data
spark.sql("""
INSERT INTO sample_table VALUES
(1, 'A', '2025-06-16 10:00:00'),
(2, 'B', '2025-06-16 11:00:00')
""")



In [0]:
# Create a materialized view
spark.sql("""
CREATE OR REPLACE MATERIALIZED VIEW sample_materialized_view
AS
SELECT * FROM sample_table
""")



In [0]:
# Function to refresh the materialized view with incremental changes
def refresh_materialized_view():
    # Get the last refresh timestamp
    last_refresh = spark.sql("SELECT MAX(updated_at) FROM sample_materialized_view").collect()[0][0]
    
    # Insert only the new changes into the materialized view
    spark.sql(f"""
    INSERT INTO sample_materialized_view
    SELECT * FROM sample_table
    WHERE updated_at > '{last_refresh}'
    """)



In [0]:
# Insert incremental data
spark.sql("""
INSERT INTO sample_table VALUES
(3, 'C', '2025-06-17 12:00:00')
""")



In [0]:
# Refresh the materialized view to include only the new changes
refresh_materialized_view()

# Display the materialized view
display(spark.sql("SELECT * FROM sample_materialized_view"))